<a href="https://colab.research.google.com/github/eou-of/stable-diffusion-webui/blob/master/nightly/absolutereality_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "cells": [
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "d4825c42"
      },
      "outputs": [],
      "source": [
        "import os\n",
        "os.environ[\"MPLBACKEND\"] = \"Agg\"\n",
        "\n",
        "import matplotlib\n",
        "matplotlib.use(\"Agg\")\n",
        "\n",
        "# === Python 3.10 설치 & 기본 설정 === (Colab 기본이 3.10이므로 스킵 가능)\n",
        "# !sudo apt-get update -y\n",
        "# !sudo apt-get install python3.10 python3.10-distutils python3.10-dev -y\n",
        "# !curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10\n",
        "\n",
        "# Python 기본 버전 3.10으로 전환 (이미 3.10이면 스킵)\n",
        "import sys\n",
        "if sys.version_info[0:2] != (3, 10):\n",
        "    !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2\n",
        "    !sudo update-alternatives --set python3 /usr/bin/python3.10\n",
        "\n",
        "# pip 최신화 (항상 실행)\n",
        "!python3 -m pip install --upgrade pip setuptools wheel\n",
        "\n",
        "# === 기존 가상환경(venv) 삭제 === (존재 여부 확인 후 삭제)\n",
        "import os\n",
        "venv_path = '/content/ui/venv'\n",
        "if os.path.exists(venv_path):\n",
        "    !rm -rf {venv_path}\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "jzdTBDKMKVo9"
      },
      "outputs": [],
      "source": [
        "import matplotlib\n",
        "matplotlib.use('Agg')\n",
        "\n",
        "#@markdown ## 구글드라이브 연동\n",
        "#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**\n",
        "Google_Drive = True #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**\n",
        "#@markdown <div><font color=\"red\">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>\n",
        "#@markdown <div><font color=\"red\">대신 한번받으면 다음 실행부터는 빨라져요</div>\n",
        "Checkpoint_Google_Save = True #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **WebUI Version**\n",
        "TagVersion = 'v1.10.1' #@param [\"v1.3.2\", \"v1.4.1\", \"v1.5.2\", \"v1.6.0\", \"v1.7.0\", \"v1.8.0\", \"v1.9.0\", \"v1.10.0\", \"v1.10.1\", \"forge\"]\n",
        "\n",
        "#@markdown -----\n",
        "#@markdown *터널링*\n",
        "\n",
        "#@markdown **Ngrok**\n",
        "Ngrok_Key = '' #@param {type:\"string\"}\n",
        "\n",
        "#@markdown **Localtunnel**\n",
        "Localtunnel = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown -----\n",
        "#@markdown *아래는 설정할 필요 없어요*\n",
        "\n",
        "#@markdown **구글드라이브에 연결 폴더**\n",
        "Google_Drive_Dir = 'webui' #@param {type:\"string\"}\n",
        "\n",
        "#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요\n",
        "#@markdown * webui/output : 생성된 이미지들이 저장되요\n",
        "#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요\n",
        "#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요\n",
        "#@markdown * webui/lycoris : 라이코리스(LyCORIS)를 넣어주면 읽어올수 있어요\n",
        "#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요\n",
        "#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요\n",
        "#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요\n",
        "\n",
        "#@markdown **초기모델(checkpoint) URL**\n",
        "Checkpoint_Url = '' #@param {type:\"string\"}\n",
        "#@markdown **초기모델 파일명**\n",
        "Checkpoint_Filename = '' #@param {type:\"string\"}\n",
        "\n",
        "#@markdown -----\n",
        "#@markdown *추가익스텐션*\n",
        "\n",
        "#@markdown **ControlNet을 사용하지 않으면 체크를 해제해주세요**\n",
        "#@markdown > *openpose, lineart, softedge, depth, canny, tile, ip2p*\n",
        "ControlNet = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown > *temporalnet, segment, inpaint, scribble, mlsd, normalbae, suffle*\n",
        "ControlNet_Extra = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **IP-Adapter 모델 다운로드**\n",
        "CotnrolNet_IPAdapter = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **T2I Adapter 모델 다운로드**\n",
        "#@markdown > *style, color, sketch, zoedepth, recolor*\n",
        "ControlNet_T2I = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **After Detailer(adetailer)를 사용하지 않으면 체크를 해제해주세요**\n",
        "ADetailer = True #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**\n",
        "DDetailer = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **Segment Anything을 사용하지 않으면 체크를 해제해주세요**\n",
        "SegmentAnything = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **AnimateDiff 사용 (WebUI 버전 v1.6.0만 지원)**\n",
        "AnimateDiff = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **AnimateDiff 추가 모션 모델 (HumansMotion,3DMotion)**\n",
        "AnimateDiff_Extra = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **mov2mov 사용**\n",
        "Mov2mov = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **Deforum 사용**\n",
        "Deforum = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **Reactor 사용**\n",
        "Reactor = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **Temporal Kit 사용**\n",
        "TemporalKit = False #@param {type:\"boolean\"}\n",
        "\n",
        "#@markdown **TemporalNet Util 사용**\n",
        "TemporalNetUtil = False #@param {type:\"boolean\"}\n",
        "\n",
        "Workspace = 'ui'\n",
        "NotebookVersion = 'nightly'\n",
        "\n",
        "if Google_Drive:\n",
        "  from google.colab import drive\n",
        "  drive.mount('/content/drive')\n",
        "\n",
        "Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion'\n",
        "if Google_Drive and Checkpoint_Google_Save:\n",
        "  Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion/google'\n",
        "\n",
        "!apt -y install -qq aria2\n",
        "\n",
        "%cd /content\n",
        "!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/install_{NotebookVersion}.sh -O /content/install.sh\n",
        "!bash /content/install.sh {Workspace} {TagVersion} {ControlNet} {DDetailer} {ADetailer} {SegmentAnything} {Mov2mov} {AnimateDiff} {Deforum} {Reactor}\n",
        "\n",
        "# 구글드라이브 연결\n",
        "!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/link_google_drive.sh -O /content/link_google_drive.sh\n",
        "!bash /content/link_google_drive.sh {Workspace} {Google_Drive} {Google_Drive_Dir} {NotebookVersion}\n",
        "\n",
        "# 커스터마이징\n",
        "%cd /content/{Workspace}\n",
        "!bash /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/install.sh {Workspace}\n",
        "if Google_Drive:\n",
        "  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/config.json /content/{Workspace}/config.json\n",
        "  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/ui-config.json /content/{Workspace}/ui-config.json\n",
        "  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/styles.csv /content/{Workspace}/styles.csv\n",
        "%cd /content/{Workspace}\n",
        "\n",
        "# checkpoint select\n",
        "!sed -i -e 's/\"sd_model_checkpoint\": \"\",/\"sd_model_checkpoint\": \"{Checkpoint_Filename}\",/g' /content/{Workspace}/config.json\n",
        "\n",
        "# upscaler\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d ./models/ESRGAN -o 4x-UltraSharp.pth\n",
        "\n",
        "# checkpoint\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \"{Checkpoint_Url}\" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}\n",
        "\n",
        "# vae\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d ./models/VAE -o kl-f8-anime2.ckpt\n",
        "\n",
        "# lora\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \"https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16\" -d ./models/Lora -o blindbox_V1Mix.safetensors\n",
        "\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \"https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/resolve/main/pytorch_lora_weights.safetensors\" -d ./models/Lora -o lcm-lora-sd15.safetensors\n",
        "\n",
        "# embeddings\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d ./embeddings -o EasyNegative.pt\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d ./embeddings -o badhandv4.pt\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d ./embeddings -o bad-hands-5.pt\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \"https://civitai.com/api/download/models/60938?type=Negative&format=Other\" -d ./embeddings/ -o negative_hand.pt\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \"https://civitai.com/api/download/models/60095?type=Negative&format=Other\" -d ./embeddings/ -o bad_prompt_version2.pt\n",
        "!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \"https://civitai.com/api/download/models/5637?type=Model&format=PickleTensor&size=full&fp=fp16\" -d ./embeddings/ -o ng_deepnegative_v1_75t.pt\n",
        "\n",
        "if ControlNet:\n",
        "  ControlNet_Model_Path = \"./extensions/controlnet/models\"\n",
        "  if TagVersion==\"forge\":\n",
        "    ControlNet_Model_Path = \"./models/ControlNet\"\n",
        "\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_openpose_fp16.safetensors\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_lineart_fp16.safetensors\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_softedge_fp16.safetensors\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1p_sd15_depth_fp16.safetensors\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_canny_fp16.safetensors\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1e_sd15_tile_fp16.safetensors\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_ip2p_fp16.safetensors\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hr16/ControlNet-HandRefiner-pruned/resolve/main/control_sd15_inpaint_depth_hand_fp16.safetensors -d {ControlNet_Model_Path} -o control_sd15_inpaint_depth_hand_fp16.safetensors\n",
        "\n",
        "  if ControlNet_Extra:\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_seg_fp16.safetensors\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15s2_lineart_anime_fp16.safetensors\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_inpaint_fp16.safetensors\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_scribble_fp16.safetensors\n",
        "\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_mlsd_fp16.safetensors\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_normalbae_fp16.safetensors\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_shuffle_fp16.safetensors\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d {ControlNet_Model_Path} -o diff_control_sd15_temporalnet_fp16.safetensors\n",
        "\n",
        "  if CotnrolNet_IPAdapter:\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter_sd15.pth\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-plus_sd15.pth\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-plus-face_sd15.pth\n",
        "\n",
        "  if ControlNet_T2I:\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_style_sd14v1.pth\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_color_sd14v1.pth\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d {ControlNet_Model_Path} -o t2iadapter_sketch_sd15v2.pth\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d {ControlNet_Model_Path} -o t2iadapter_zoedepth_sd15v1.pth\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors -d {ControlNet_Model_Path} -o ioclab_sd15_recolor.safetensors\n",
        "\n",
        "if AnimateDiff_Extra:\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/174464 -d ./extensions/animatediff/model -o improvedHumansMotion_refinedHumanMovement.ckpt\n",
        "  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/178017 -d ./extensions/animatediff/model -o improved3DMotion_improved3DV1.ckpt\n",
        "\n",
        "if TemporalKit:\n",
        "  !git clone https://github.com/CiaraStrawberry/TemporalKit ./extensions/TemporalKit\n",
        "  !pip install scenedetect\n",
        "\n",
        "if TemporalNetUtil:\n",
        "  !git clone https://github.com/ninjaneural/temporalnet_util ./extensions/temporalnet_util\n",
        "  !pip install ffmpeg-python\n",
        "  if ControlNet:\n",
        "    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d {ControlNet_Model_Path} -o diff_control_sd15_temporalnet_fp16.safetensors\n",
        "\n",
        "if Localtunnel:\n",
        "  !npm install -g localtunnel\n",
        "\n",
        "  import subprocess\n",
        "  import threading\n",
        "  import time\n",
        "  import socket\n",
        "  import urllib.request\n",
        "\n",
        "  def iframe_thread(port):\n",
        "    while True:\n",
        "        time.sleep(0.5)\n",
        "        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\n",
        "        result = sock.connect_ex(('127.0.0.1', port))\n",
        "        if result == 0:\n",
        "          break\n",
        "        sock.close()\n",
        "    print(\"\\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\\n\")\n",
        "\n",
        "    print(\"The password/enpoint ip for localtunnel is:\", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip(\"\\n\"))\n",
        "    p = subprocess.Popen([\"lt\", \"--port\", \"{}\".format(port)], stdout=subprocess.PIPE)\n",
        "    for line in p.stdout:\n",
        "      print(line.decode(), end='')\n",
        "\n",
        "\n",
        "  threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()\n",
        "\n",
        "models_dir = f'/content/drive/MyDrive/{Google_Drive_Dir}/models' if Google_Drive else ''\n",
        "models_arg = f'--models-dir {models_dir}' if models_dir else ''\n",
        "\n",
        "if Ngrok_Key:\n",
        "  !python launch.py --xformers --no-half-vae --theme dark --ngrok {Ngrok_Key} --skip-load-model-at-start --precision half --lowvram {models_arg}\n",
        "else:\n",
        "  !python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access --theme dark --gradio-queue --skip-load-model-at-start --precision half --lowvram {models_arg}\n"
      ]
    }
  ],
  "metadata": {
    "accelerator": "GPU",
    "colab": {
      "private_outputs": true,
      "provenance": []
    },
    "gpuClass": "standard",
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.10.11"
    },
    "vscode": {
      "interpreter": {
        "hash": "32c23ec9c124fdbef417fa7c12f036622d7e14abb755fc2527a38410604ea1a4"
      }
    }
  },
  "nbformat": 4,
  "nbformat_minor": 0
}